In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import os

# Setup headless Chrome
options = Options()
options.headless = True
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def get_all_product_links(base_url, total_pages=4):
    all_links = []
    for page in range(1, total_pages + 1):
        url = f"{base_url}?page={page}"
        driver.get(url)
        time.sleep(3)  # Wait for JS to load
        links = []
        for a in driver.find_elements(By.CSS_SELECTOR, 'a[href*="/product-page/"]'):
            href = a.get_attribute('href')
            if href and href not in all_links:
                all_links.append(href)
    return all_links

def get_product_links(collection_url):
    driver.get(collection_url)
    time.sleep(3)  # Wait for JS to load
    links = []
    # Find all product links (update selector as needed)
    for a in driver.find_elements(By.CSS_SELECTOR, 'a[href*="/product-page/"]'):
        href = a.get_attribute('href')
        if href and href not in links:
            links.append(href)
    return links

def get_product_image(product_url):
    driver.get(product_url)
    time.sleep(5)  # Wait for images to load and unblur
    # Find the main product image (update selector as needed)
    images = driver.find_elements(By.CSS_SELECTOR, 'img[src*="static.wixstatic.com/media/"]')
    # Optionally, filter by size or alt attribute
    for img in images:
        src = img.get_attribute('src')
        alt = img.get_attribute('alt')
        if src and alt and "Dress" in alt:  # crude filter, adjust as needed
            return src
    return None

def download_image(url, filename):
    import requests
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)

# Main
collection_url = 'https://www.omarmansoor.com/all-collections'
product_links = get_all_product_links(collection_url, total_pages=4)
os.makedirs('images', exist_ok=True)
for link in product_links:
    img_url = get_product_image(link)
    if img_url:
        fname = os.path.join('images', link.split('/')[-1] + '.jpg')
        download_image(img_url, fname)
        print(f'Downloaded {fname}')
driver.quit()

Downloaded images/zhui-feng-dress.jpg


KeyboardInterrupt: 